<a href="https://colab.research.google.com/github/VishnuSaiKarthikGindi/MyTripBuddy/blob/main/Multi_AI_Agent_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain_community langchain_core tiktoken langchainhub langchain langchain_openai openai langgraph pinecone-client googlemaps langchain_pinecone pydantic

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_toolkits import AmadeusToolkit
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain_community.tools import OpenWeatherMapQueryRun
import googlemaps
from langchain.llms import OpenAI
from langchain.tools import tool
from bs4 import BeautifulSoup
from langchain.tools.base import BaseTool
import requests

In [5]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.embeddings import OpenAIEmbeddings
import pinecone
import os

from dotenv import load_dotenv
load_dotenv()

True

In [7]:
FAQs = {
    "Airlines_FAQs": ["https://www.aa.com/i18n/customer-service/faqs/customer-service-faqs.jsp",
                        "https://www.delta.com/us/en/travel-planning-center/travel-planning-faqs?srsltid=AfmBOoq5-LF55hW4-i9LORhmYi80_zTM4DBbTSz-mZKiAeYQsGsPwkvk",
                        "https://www.united.com/en/us/fly/help-center.html",
                        "https://faq.flyfrontier.com/help"],
    "Hotel_FAQs": [ "https://www.expedia.com/helpcenter/?pwaDialog=article-dialog&product=Lodging&productId=lodging",
                 "https://www.wyndhamhotels.com/wyndham/contact-us/faq",
                  "https://www.booking.com/tpi_faq.html"
                ]
    }

In [4]:
embeddings = OpenAIEmbeddings(openai_api_key = os.environ["OPENAI_API_KEY"])

<ipython-input-4-b082dd00b321>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key = os.environ["OPENAI_API_KEY"])


In [10]:
class LangGraphVectorStore:
    def __init__(self, pinecone_api_key: str, pinecone_environment: str, index_name: str):
        # Initialize Pinecone
        pc = Pinecone(api_key=pinecone_api_key)
        self.index_name = index_name
        existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

        if self.index_name not in existing_indexes:
          pc.create_index(
                        name=index_name,
                        dimension=1536,
                        metric="cosine",
                        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
                      )
        index = pc.Index(index_name)

        # Initialize embeddings
        self.embeddings = OpenAIEmbeddings(openai_api_key = os.environ["OPENAI_API_KEY"])
        self.vectorstore = PineconeVectorStore(index=index, embedding=self.embeddings)

    def load_and_split_docs(self, urls):
        # Load documents
        docs = [WebBaseLoader(url).load() for url in urls]
        docs_list = [item for sublist in docs for item in sublist]

        # Split documents
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=500, chunk_overlap=0
        )
        return text_splitter.split_documents(docs_list)

    def add_documents(self, urls):
        # Load and split documents
        doc_splits = self.load_and_split_docs(urls)

        # Add documents to the vector store
        self.vectorstore.add_documents(doc_splits)
        print(f"Inserted {len(doc_splits)} documents.")
        return doc_splits

    def create_index_wrapper(self):
        # Create a wrapper around the vector store
        return VectorStoreIndexWrapper(vectorstore=self.vectorstore)

    def retrieve_documents(self):
        # Get retriever
        return self.vectorstore.as_retriever()

# Example usage
if __name__ == "__main__":
    # Initialize the LangGraphVectorStore with Pinecone credentials
    pinecone_api_key = os.getenv("PINECONE_API_KEY")
    pinecone_environment = "aws-starter"
    index_name = "mytripbuddy"

    lang_graph = LangGraphVectorStore(
        pinecone_api_key=pinecone_api_key,
        pinecone_environment=pinecone_environment,
        index_name=index_name
    )

    # Add Airline FAQs
    print("Indexing Airline FAQs...")
    airline_docs = lang_graph.add_documents(FAQs["Airlines_FAQs"])

    # Add Hotel FAQs
    print("Indexing Hotel FAQs...")
    hotel_docs = lang_graph.add_documents(FAQs["Hotel_FAQs"])

    # Create index wrapper and retriever
    vector_index = lang_graph.create_index_wrapper()
    retriever = lang_graph.retrieve_documents()

<ipython-input-10-7c83e92ca316>:18: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  self.embeddings = OpenAIEmbeddings(openai_api_key = os.environ["OPENAI_API_KEY"])


Indexing Airline FAQs...
Inserted 33 documents.
Indexing Hotel FAQs...
Inserted 21 documents.


In [ ]:
retriever.invoke("What is cancellation policy for hotel?")

In [7]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents.agent_types import AgentType
from langchain.schema import AgentAction, AgentFinish
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
import requests
import os

class TripAdvisorAPI:
    def __init__(self, api_key, base_url="https://api.content.tripadvisor.com/api/v1"):
        """
        Initialize the API client.

        :param api_key: The API key for accessing TripAdvisor endpoints.
        :param base_url: The base URL for the TripAdvisor API.
        """
        self.api_key = api_key
        self.base_url = base_url

    def _make_request(self, endpoint, params):
        """
        Helper method to make API requests.

        :param endpoint: The API endpoint to call.
        :param params: Query parameters for the request.
        :return: JSON response or None in case of failure.
        """
        headers = {"Accept": "application/json"}
        try:
            response = requests.get(f"{self.base_url}/{endpoint}", headers=headers, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"API Request failed: {e}")
            return None

    def search_location(self, search_query, category=None, phone=None, address=None, lat_long=None, radius=None, radius_unit=None, language="en"):
        """
        Search for a location by name, category, phone, or coordinates.

        :param search_query: Name of the location to search for.
        :param category: Optional filter by category (e.g., hotels, attractions).
        :param phone: Optional filter by phone number.
        :param address: Optional filter by address.
        :param lat_long: Optional latitude/longitude pair (e.g., "42.3455,-71.10767").
        :param radius: Optional radius for filtering results.
        :param radius_unit: Unit for radius (e.g., km, mi).
        :param language: Language for the results (default is "en").
        :return: JSON response containing location search results.
        """
        endpoint = "location/search"
        params = {
            "key": self.api_key,
            "searchQuery": search_query,
            "category": category,
            "phone": phone,
            "address": address,
            "latLong": lat_long,
            "radius": radius,
            "radiusUnit": radius_unit,
            "language": language
        }
        return self._make_request(endpoint, params)

    def get_location_details(self, location_id, language="en", currency="USD"):
        """
        Get detailed information about a specific location.

        :param location_id: The unique TripAdvisor location ID.
        :param language: Language for the results (default is "en").
        :param currency: Currency for the results (default is "USD").
        :return: JSON response containing location details.
        """
        endpoint = f"location/{location_id}/details"
        params = {
            "key": self.api_key,
            "language": language,
            "currency": currency
        }
        return self._make_request(endpoint, params)

    def nearby_search(self, lat_long, category=None, radius=None, radius_unit="km", language="en"):
        """
        Search for nearby locations based on latitude/longitude.

        :param lat_long: Latitude/Longitude pair (e.g., "42.3455,-71.10767").
        :param category: Optional filter by category (e.g., hotels, attractions).
        :param radius: Optional radius for filtering results.
        :param radius_unit: Unit for radius (default is "km").
        :param language: Language for the results (default is "en").
        :return: JSON response containing nearby search results.
        """
        endpoint = "location/nearby_search"
        params = {
            "key": self.api_key,
            "latLong": lat_long,
            "category": category,
            "radius": radius,
            "radiusUnit": radius_unit,
            "language": language
        }
        return self._make_request(endpoint, params)

In [30]:
from typing import Dict, Any, Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
import ast
import regex as re

# Define Pydantic schema for tool arguments
class SearchAccommodationAndAttractionsArgs(BaseModel):
    query: str = Field(..., description="Natural language query for accommodations or attractions.")

# Define the tool class
class SearchAccommodationAndAttractions(BaseTool):
    name: str = "SearchAccommodationAndAttractions"
    description: str = (
                "Search for hotels, attractions, and restaurants using the TripAdvisor API. "
                "Input should be a dictionary with keys: 'location' (required), 'type' (required: 'hotels', 'attractions', or 'restaurants'), "
                "and optionally 'radius', 'radius_unit', and 'language'. Returns relevant information about the specified type near the given location."
    )

    args_schema: Type[BaseModel] = SearchAccommodationAndAttractionsArgs
    api_client: TripAdvisorAPI  # Declare API client as a field

    def _run(self, query: str) -> Dict[str, Any]:
      def parse_user_query(query: str) -> Dict[str, Any]:
          parsed_query = {"language": "en"}

          try:
              # Attempt to parse the query as a dictionary-like string
              query_dict = ast.literal_eval(query)
              if isinstance(query_dict, dict):
                  parsed_query["searchQuery"] = query_dict.get("location", "")
                  parsed_query["category"] = query_dict.get("type", "")
                  parsed_query["radius"] = query_dict.get("radius", None)
                  parsed_query["radiusUnit"] = query_dict.get("radiusUnit", None)
                  return parsed_query
          except (ValueError, SyntaxError):
              # If the query is not a dictionary-like string, proceed with natural language parsing
              pass

          # Define patterns for different query types
          patterns = {
              "top_attractions": r"top attractions in (?P<location>.+)",
              "must_visit": r"must-visit places in (?P<location>.+)",
              "best_things_to_do": r"best things to do in (?P<location>.+) for (?P<type>families|couples|solo travelers)",
              "budget_friendly": r"free or budget-friendly attractions in (?P<location>.+)",
              "hidden_gems": r"hidden gems in (?P<location>.+)",
              "cultural_landmarks": r"cultural or historical landmarks in (?P<location>.+)",
              "outdoor_activities": r"outdoor activities or natural attractions in (?P<location>.+)",
              "best_time_to_visit": r"best time to visit (?P<attraction>.+)",
              "crowded_times": r"is (?P<attraction>.+) crowded during weekends/holidays",
              "best_season": r"best season to visit (?P<location>.+)"
          }

          # Extract information using regex patterns
          for key, pattern in patterns.items():
              match = re.search(pattern, query, re.IGNORECASE)
              if match:
                  parsed_query.update(match.groupdict())
                  if key in ["best_things_to_do", "cultural_landmarks", "outdoor_activities"]:
                      parsed_query["category"] = "attractions"
                  elif key == "budget_friendly":
                      parsed_query["category"] = "budget"
                  return parsed_query

          tokens = query.lower().split()
          categories = {"hotels", "attractions", "restaurants"}

          # Extract category from the query
          for category in categories:
              if category in tokens:
                  parsed_query["category"] = category
                  break

          # Extract location or POI
          if "near" in tokens:
              idx = tokens.index("near") + 1
              if idx < len(tokens):
                  parsed_query["searchQuery"] = " ".join(tokens[idx:])

          # Extract radius if mentioned
          if "within" in tokens:
              idx = tokens.index("within") + 1
              try:
                  parsed_query["radius"] = int(tokens[idx])
                  if idx + 1 < len(tokens) and tokens[idx + 1] in {"km", "mi"}:
                      parsed_query["radiusUnit"] = tokens[idx + 1]
              except (ValueError, IndexError):
                  pass

          return parsed_query

      query_params = parse_user_query(query)
      if "latLong" in query_params:
          results = self.api_client.nearby_search(
              lat_long=query_params.get("latLong"),
              category=query_params.get("category"),
              radius=query_params.get("radius"),
              radius_unit=query_params.get("radiusUnit"),
              language=query_params.get("language"),
          )
      else:
          results = self.api_client.search_location(
                search_query=query_params.get("searchQuery"),
                category=query_params.get("category"),
                radius=query_params.get("radius"),
                radius_unit=query_params.get("radiusUnit"),
                language=query_params.get("language"),
          )
      return results

    def _arun(self, query: str) -> Dict[str, Any]:
        raise NotImplementedError("This tool does not support async")

In [ ]:
# Initialize the Amadeus Toolkit
toolkit = AmadeusToolkit()
amadeus_tool = toolkit.get_tools()

# Initialize OpenWeatherMap Wrapper
weather_wrapper = OpenWeatherMapAPIWrapper(
    top_k_results=1, doc_content_chars_max=200
)
weather_tool = Tool(
    name="WeatherQuery",
    description="Fetch weather information for a specific location using OpenWeatherMap.",
    func=weather_wrapper.run
)

# Initialize Google Maps for routes
gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAPS_API_KEY"))

def get_route(origin: str, destination: str) -> str:
    """
    Fetches driving route from Google Maps.
    Args:
        origin (str): The starting location.
        destination (str): The destination location.
    Returns:
        str: A text-based driving itinerary.
    """
    directions = gmaps.directions(origin, destination, mode="driving")
    if directions:
        steps = directions[0]["legs"][0]["steps"]
        itinerary = "Driving Route:\n"
        for step in steps:
            itinerary += f"{step['html_instructions']} for {step['distance']['text']}\n"
        return itinerary
    return "No route found."

route_tool = Tool(
    name="RouteQuery",
    description="Fetch driving directions from one location to another.",
    func=lambda x: get_route(x['origin'], x['destination'])
)

In [ ]:
# Tools for the travel agent
tools = [amadeus_tool, weather_tool, route_tool]

# Initialize TripAdvisor API tool
tripadvisor_api = TripAdvisorAPI(api_key=os.getenv("TRIPADVISOR_API_KEY"))
search_tool = SearchAccommodationAndAttractions(api_client=tripadvisor_api)
tools.append(search_tool)

# Set up the turbo LLM
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

# Integrate tools with a React agent
agent = create_react_agent(
    llm,
    tools,
    prompt=ChatPromptTemplate.from_messages(
        [
            ("system", "You are a travel assistant."),
            ("human", "{input}"),
        ]
    )
)

# Initialize vectorstore retriever
vectorstore_retriever = lang_graph.retrieve_documents()

In [ ]:
### Data model ###
class RouteQuery(BaseModel):
    """
    Route a user query to the most relevant datasource.
    """
    datasource: Literal["tools", "vectorstore"] = Field(
        ...,
        description="Route user questions to tools or the vectorstore agent."
    )

### Routing ###
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
        You are an expert at routing a user question to tools or vectorstore.
        Use tools for queries related to flights, weather, routes, accommodations, and attractions.
        Use the vectorstore for indexed topics such as FAQs and technical concepts.
        """),
        ("human", "{question}"),
    ]
)

def route_question(state):
    """
    Routes the user's question to the appropriate agent.
    """
    question = state["question"]
    result = route_prompt.invoke({"question": question})
    if result.datasource == "tools":
        return "tools_agent"
    return "vectorstore"

def retrieve_documents(state):
    """
    Retrieves documents from the vectorstore.
    """
    question = state["question"]
    documents = vectorstore_retriever.retrieve_documents(question)
    return {"documents": documents}

def execute_tools(state):
    """
    Executes tools for query resolution.
    """
    question = state["question"]
    result = agent.run(question)
    return {"generation": result}

In [ ]:
### Workflow ###
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(GraphState)

workflow.add_conditional_edges(
    START,
    route_question,
    {"tools_agent": "tools_agent", "vectorstore": "vectorstore"},
)

workflow.add_node("tools_agent", execute_tools)
workflow.add_node("vectorstore", retrieve_documents)

workflow.add_edge("tools_agent", END)
workflow.add_edge("vectorstore", END)

app = workflow.compile()

### Running Queries ###
inputs = {"question": "What are the best attractions in Paris?"}
for output in app.stream(inputs):
    print(output)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass